In [1]:
import os
import re
from dotenv import load_dotenv
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from typing import List, Any, Optional, Dict
from rich.pretty import pprint

def pretty_print(title: str=None, content: Any=None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

from IPython.display import display, HTML, Markdown

In [2]:
from typing import Any, Optional, List
from rich.pretty import pprint

def pretty_print(title: Optional[str] = None, content: Any = None, exclude: Optional[List[str]] = None):
    """
    Prints the content prettily. If a title is provided, it prints the title before the content.
    Can exclude specified attributes from being printed if content is a dictionary.

    Args:
        title (Optional[str]): The title to print before the content. Defaults to None.
        content (Any): The content to be printed.
        exclude (Optional[List[str]]): List of keys to exclude from the content if it's a dictionary. Defaults to None.
    """
    if exclude is not None and isinstance(content, dict):
        content = {k: v for k, v in content.items() if k not in exclude}
    
    if title is not None:
        print(title)
    
    pprint(content)

In [3]:
load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

In [4]:
df = pd.read_parquet("./data/splade_embeds.parquet")
# Convert column names to snake_case for compatibility with LanceDB
original_columns = df.columns
snake_case_columns = {col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in original_columns}
df.rename(columns=snake_case_columns, inplace=True)
df.head(1)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings
0,1078,1575952538,https://www.reddit.com/r/legaladvice/comments/...,e8lsen,I applied for a job and after two interviews I...,"Failed a drug test due to amphetamines, I have...",employment,5,"[9.475638042064453e-05, 0.0005111666301983955,...",493,"""Validity of Schedule II Drug Prescription in ...",PR,8,Employment Legal Concerns and Issues,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [6]:
from src.search.models import dataframe_to_text_nodes

nodes = dataframe_to_text_nodes(df=df, id_column='index', text_col='body', metadata_fields=['state', 'full_link'], embedding_column='embeddings')

In [7]:
pretty_print(nodes[1])

Node ID: 2098
Text: Hi everyone, thanks in advance for any guidance.  I was driving
home yesterday and managed to get pulled over outside Chattanooga
going 21 miles over the speed limit (76 in a 55).  It was unfortunate,
right at the part of the highway where the speed limit drops from 70
to 55 because of the mountains, and I just didn't adjust in time.  I
usually ...


None

In [17]:
from typing import Tuple
import numpy as np
from typing import cast
from llama_index.core.vector_stores.types import VectorStore
from llama_index.core.vector_stores import MetadataFilters, MetadataFilter
from llama_index.core.vector_stores import (
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index.core.schema import BaseNode


def get_top_k_embeddings(
    query_embedding: List[float],
    doc_embeddings: List[List[float]],
    doc_ids: List[str],
    similarity_top_k: int = 5,
) -> Tuple[List[float], List]:
    """Get top nodes by similarity to the query."""
    # dimensions: D
    qembed_np = np.array(query_embedding)
    # dimensions: N x D
    dembed_np = np.array(doc_embeddings)
    # dimensions: N
    dproduct_arr = np.dot(dembed_np, qembed_np)
    # dimensions: N
    norm_arr = np.linalg.norm(qembed_np) * np.linalg.norm(
        dembed_np, axis=1, keepdims=False
    )
    # dimensions: N
    cos_sim_arr = dproduct_arr / norm_arr

    # now we have the N cosine similarities for each document
    # sort by top k cosine similarity, and return ids
    tups = [(cos_sim_arr[i], doc_ids[i]) for i in range(len(doc_ids))]
    sorted_tups = sorted(tups, key=lambda t: t[0], reverse=True)

    sorted_tups = sorted_tups[:similarity_top_k]

    result_similarities = [s for s, _ in sorted_tups]
    result_ids = [n for _, n in sorted_tups]
    return result_similarities, result_ids


def filter_nodes(nodes: List[BaseNode], filters: MetadataFilters):
    filtered_nodes = []
    for node in nodes:
        matches = True
        for f in filters.filters:
            if f.key not in node.metadata:
                matches = False
                continue
            if f.value != node.metadata[f.key]:
                matches = False
                continue
        if matches:
            filtered_nodes.append(node)
    return filtered_nodes


def dense_search(query: VectorStoreQuery, nodes: List[BaseNode]):
    """Dense search."""
    query_embedding = cast(List[float], query.query_embedding)
    doc_embeddings = [n.embedding for n in nodes]
    doc_ids = [n.node_id for n in nodes]
    return get_top_k_embeddings(
        query_embedding,
        doc_embeddings,
        doc_ids,
        similarity_top_k=query.similarity_top_k,
    )

In [9]:
class BaseVectorStore(VectorStore):
    """Simple custom Vector Store.

    Stores documents in a simple in-memory dict.

    """
    stores_text: bool = True
    
    def __init__(self) -> None:
        """Init params."""
        self.node_dict: Dict[str, BaseNode] = {}

    def get(self, text_id: str) -> List[float]:
        """Get embedding."""
        return self.node_dict[text_id]

    def add(
        self,
        nodes: List[BaseNode],
    ) -> List[str]:
        """Add nodes to index."""
        for node in nodes:
            self.node_dict[node.node_id] = node

    def delete(self, node_id: str, **delete_kwargs: Any) -> None:
        """
        Delete nodes using with node_id.

        Args:
            node_id: str

        """
        del self.node_dict[node_id]

    def query(
        self,
        query: VectorStoreQuery,
        **kwargs: Any,
    ) -> VectorStoreQueryResult:
        """Get nodes for response."""

        query_embedding = cast(List[float], query.query_embedding)
        doc_embeddings = [n.embedding for n in self.node_dict.values()]
        doc_ids = [n.node_id for n in self.node_dict.values()]

        similarities, node_ids = get_top_k_embeddings(
            query_embedding,
            doc_embeddings,
            doc_ids,
            similarity_top_k=query.similarity_top_k,
        )
        result_nodes = [self.node_dict[node_id] for node_id in node_ids]

        return VectorStoreQueryResult(
            nodes=result_nodes, similarities=similarities, ids=node_ids
        )

    def persist(self, persist_path, fs=None) -> None:
        """Persist the SimpleVectorStore to a directory.

        NOTE: we are not implementing this for now.

        """
        pass

In [10]:
vector_store = BaseVectorStore()
vector_store.add(nodes)

In [12]:
print(str(nodes[1]))

Node ID: 2098
Text: Hi everyone, thanks in advance for any guidance.  I was driving
home yesterday and managed to get pulled over outside Chattanooga
going 21 miles over the speed limit (76 in a 55).  It was unfortunate,
right at the part of the highway where the speed limit drops from 70
to 55 because of the mountains, and I just didn't adjust in time.  I
usually ...


In [13]:
import openai

client = openai.OpenAI()

In [14]:
query_str = "Can you tell me about medicinal marijuana use at work?"

query_embedding = client.embeddings.create(
                    input=query_str, model='text-embedding-ada-002'
                ).data[0].embedding

In [15]:
query_obj = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

query_result = vector_store.query(query_obj)
for similarity, node in zip(query_result.similarities, query_result.nodes):
    print(
        "\n----------------\n"
        f"[Node ID {node.node_id}] Similarity: {similarity}\n\n"
        f"{node.get_content(metadata_mode='all')}"
        "\n----------------\n\n"
    )


----------------
[Node ID 8075] Similarity: 0.8717243548229164

Metadata:
state = MI
full_link = https://www.reddit.com/r/legaladvice/comments/gr4q8s/is_it_legal_in_new_york_to_discriminate_against/
----------------------------------------
Content:
I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.
----------------



----------------
[Node ID 7331] Similarity: 0.8698168024576537

Metadata:
state = PA
full_link = https://www.reddit.com/r/legaladvice/comments/e8y7gr/considering_getting_a_medical_marijuana_card/

In [24]:
filters = MetadataFilters(
    filter_condition="or",
    filters=[
        MetadataFilter(key="state", value="MI", operator="text_match")
    ]
)

query_obj = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=3, filters=filters
)

query_result = vector_store.query(query_obj)
for similarity, node in zip(query_result.similarities, query_result.nodes):
    print(
        "\n----------------\n"
        f"[Node ID {node.node_id}] Similarity: {similarity}\n\n"
        f"{node.get_content(metadata_mode='all')}"
        "\n----------------\n\n"
    )


----------------
[Node ID 8075] Similarity: 0.8717243548229164

Metadata:
state = MI
full_link = https://www.reddit.com/r/legaladvice/comments/gr4q8s/is_it_legal_in_new_york_to_discriminate_against/
----------------------------------------
Content:
I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.
----------------



----------------
[Node ID 7331] Similarity: 0.8698168024576537

Metadata:
state = PA
full_link = https://www.reddit.com/r/legaladvice/comments/e8y7gr/considering_getting_a_medical_marijuana_card/

In [25]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_vector_store(vector_store)
query_engine = vector_index.as_query_engine()

response = query_engine.query("Write a detailed summary of the legal issues involving medical marijuana")
Markdown(str(response))

The legal issues involving medical marijuana can vary depending on the state. In some cases, individuals who own a mobile home in a park may face restrictions on using medical marijuana on the park's property, even if they own their home outright. This situation raises questions about the rights of individuals to use medical marijuana in certain locations.

Additionally, discrimination against medical marijuana patients can occur in the workplace, as seen in the case of a New York State medical marijuana patient who faced discrimination during the hiring process at a hospital. This highlights the legal complexities surrounding the rights of medical marijuana patients in employment settings and the potential for discrimination based on their status as medical marijuana cardholders.

Overall, the legal issues involving medical marijuana encompass a range of areas such as property rights, employment discrimination, patient rights, and state-specific regulations governing the use of medical marijuana.

In [26]:
Markdown(response.source_nodes[0].get_content())

Asking this for a friend who lives in a mobile home park in California.  He wants to get a medical marijuana card for his medical conditions so he can get off his pain medicine, but the park is telling him that even though he owns his home 100%, because its on the park's land he can't use it or bring it on the property.  Can the park legally do this or is there some legal medical grounds for him?  

In [27]:
TEMPERATURE = 0.0
SIM_TOP_K = 6
RERANK_TOP_K = 3
WIN_SZ = 1024

import asyncio
from tqdm.asyncio import tqdm

import chromadb
from chromadb.api.models.Collection import Collection

from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.core.llms.utils import LLMType
from llama_index.core.schema import NodeWithScore
from llama_index.core.indices.base import BaseIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.vector_stores import ChromaVectorStore
from llama_index.core.base.llms.types import CompletionResponse
from llama_index.core.node_parser import SentenceWindowNodeParser

from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.legacy.core.response.schema import RESPONSE_TYPE
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.query_engine import CustomQueryEngine, BaseQueryEngine

In [28]:
li_model = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)

In [29]:
vector_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)

In [30]:
base_retriever = vector_index.as_retriever(similarity_top_k=20)

In [31]:
from collections import defaultdict


class MultiQueriesRetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, model:OpenAI):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self._retrievers = [base_retriever]
        self.base_retriever = base_retriever
        self.model = model

    @classmethod
    def flatten(cls, lst: List[List[Any]]) -> List[Any]:
        return [element for sublist in lst for element in sublist]

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    async def run_gen_queries(self,generated_queries: List[str]) -> List[NodeWithScore]:
        tasks = list(map(lambda q: self.base_retriever.aretrieve(q), generated_queries))
        res = await tqdm.gather(*tasks)
        return MultiQueriesRetriever.flatten(res)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return list()

    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query: str= query_bundle.query_str
        generated_queries: List[str] = self.gen_queries(query)
        pretty_print("generated_queries", generated_queries)
        node_with_scores = await self.run_gen_queries(generated_queries)
        # node_with_scores_uniqued = dict()
        # node_with_scores_uniqued = {node_with_score.get_content(): node_with_score for node_with_score in node_with_scores}
        # return node_with_scores_uniqued.values()
        # Fusion with ranking, ref:https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
        output_documents = self._reciprocal_rank_fusion(node_with_scores)
        return output_documents
     
    def _reciprocal_rank_fusion(self, document_lists: List[List[NodeWithScore]]) -> List[NodeWithScore]:
        """
        Merge multiple lists of NodeWithScore and assign scores based on reciprocal rank fusion.
        
        Args:
            document_lists (List[List[NodeWithScore]]): A list of lists, where each sublist contains NodeWithScore objects.
        
        Returns:
            List[NodeWithScore]: A list of NodeWithScore objects with updated scores based on reciprocal rank fusion.
        
        The constant k is set to 61 (60 was suggested by the original paper,
        plus 1 as python lists are 0-based and the paper used 1-based ranking).
        """
        k = 61

        scores_map = defaultdict(int)
        documents_map = {}
        weights = self.weights if self.weights else [1 / len(document_lists)] * len(document_lists)

        # Calculate weighted reciprocal rank fusion score
        for documents, weight in zip(document_lists, weights):
            for rank, doc in enumerate(documents):
                scores_map[doc.id] += (weight * len(document_lists)) / (k + rank)
                documents_map[doc.id] = doc

        # Normalize scores. Note: len(results) / k is the maximum possible score,
        # achieved by being ranked first in all doc lists with non-zero weight.
        for id in scores_map:
            scores_map[id] /= len(document_lists) / k

        for doc in documents_map.values():
            doc.score = scores_map[doc.id]

        return documents_map.values()



In [32]:
mr = MultiQueriesRetriever(base_retriever, li_model)

In [33]:
import nest_asyncio
nest_asyncio.apply()

ls = mr.gen_queries("Is medical marijuana allowed at work in California?")
pretty_print("ls", ls)
rls = await mr.run_gen_queries(ls)

ls


[
│   'What are the regulations regarding medical marijuana use in workplaces in California?',
│   'Can employees in California use medical marijuana during work hours?',
│   'What is the policy on medical marijuana use in the workplace in California?',
│   'Are there any restrictions on medical marijuana consumption while at work in California?',
│   'Is the usage of medical marijuana permitted in California workplaces?'
]

100%|██████████| 5/5 [00:01<00:00,  3.61it/s]


In [35]:
len(rls)

100

In [36]:
class MultiQueriers:
    def __init__(self,
                 base_retriever: BaseRetriever,
                 base_query_engine: BaseQueryEngine,
                 model:OpenAI,
                 sub_queries_in_bundle_to_answer: bool=True):
        self.base_retriever = base_retriever
        self.base_query_engine = base_query_engine
        self.model = model
        self.sub_queries_in_bundle_to_answer = sub_queries_in_bundle_to_answer
        self.gen_q_template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self.qa_prompt_template = PromptTemplate("""Here is the question you need to answer:

    \n --- \n {query_str} \n --- \n

    Here is any available background question + answer pairs:

    \n --- \n {q_a_pairs} \n --- \n

    Here is additional context relevant to the question:

    \n --- \n {context_str} \n --- \n

    Use the above context and any background question + answer pairs to answer the question: \n {query_str}
    """)

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.gen_q_template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    def query_by_retriever(self, query_str: str) ->str:
        nodes = self.base_retriever.retrieve(query_str)
        res = "\n\n".join([n.node.get_content() for n in nodes])
        return res

    async def run_gen_queries(self,generated_queries: List[str]) -> str:
        sub_query_qa_pairs = list()
        if self.sub_queries_in_bundle_to_answer:
            # Answer all queries in one bundle.
            tasks = list(map(lambda q: self.base_query_engine.aquery(q), generated_queries))
            res = await tqdm.gather(*tasks)
            for idx, (query, answer) in enumerate(zip(generated_queries, res)):
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            pretty_print("sub_query_qa_pairs", sub_query_qa_pairs)
            return "\n\n".join(sub_query_qa_pairs)
        else:
            # Answer queries in step-wise.
            # One sub-query will be answered based on the context of sub-query and
            # history of pairs of previous queries and answers.
            for idx, query in enumerate(generated_queries):
                pretty_print(f"{idx}. query", query)
                pretty_print(f"{idx}. sub_query_qa_pairs", sub_query_qa_pairs)
                context_str = self.query_by_retriever(query)
                sub_query = self.qa_prompt_template.format(
                                  query_str=query,
                                  q_a_pairs="\n\n".join(sub_query_qa_pairs),
                                  context_str=context_str
                )
                pretty_print("sub_query", sub_query)
                answer: str = self.model.complete(sub_query)
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            return "\n\n".join(sub_query_qa_pairs)

    def query(self, query_str: str) -> CompletionResponse:
        return ""

    async def aquery(self, query_str: str) -> CompletionResponse:
        generated_queries: List[str] = self.gen_queries(query_str)
        sub_query_qa_pairs: str = await self.run_gen_queries(generated_queries)
        context_str = self.query_by_retriever(query_str)
        final_query: str = self.qa_prompt_template.format(query_str=query_str,
                                q_a_pairs=sub_query_qa_pairs,
                                context_str=context_str)
        pretty_print("final_query", final_query)
        response: str = self.model.complete(final_query)
        return response

In [37]:
import nest_asyncio
nest_asyncio.apply()

query_text = """What are the legal issues involving medical marijuana and employment?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in Markdown with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=True)
final_res = await mqe.aquery(query_text)
display(Markdown(final_res.text))

100%|██████████| 5/5 [00:03<00:00,  1.25it/s]

sub_query_qa_pairs


[
│   "Question 0: 1. What legal implications exist regarding medical marijuana and workplace policies?\nAnswer: Workplace policies can supersede an individual's possession of a medical marijuana card, potentially leading to conflicts between state laws allowing medical marijuana use and company policies prohibiting it. Employers may enforce zero-tolerance drug policies, even in states where medical marijuana is legal, and this can impact an employee's job status. Additionally, there may be legal implications related to discrimination against medical marijuana patients in the hiring process, as seen in the case of the individual in New York who was refused employment after disclosing their medical marijuana card.\n",
│   'Question 1: 2. How does medical marijuana affect employment laws?\nAnswer: Medical marijuana can impact employment laws differently depending on the state. It is important to consider the specific regulations in each state regarding medical marijuana use, drug testing policies, and workplace discrimination laws. Employers may have varying approaches to medical marijuana use by employees, especially in relation to drug testing, hiring decisions, and workplace policies.\n',
│   'Question 2: 3. What are the current debates on using medical marijuana in the workplace?\nAnswer: The current debates on using medical marijuana in the workplace revolve around the conflict between state laws legalizing medical marijuana and company policies that maintain zero tolerance drug policies. Issues such as whether having a medical marijuana card protects an employee from repercussions at work, the impact on safety-sensitive positions, and the potential for discrimination against medical marijuana patients in the hiring process are key points of contention in these debates. Additionally, concerns about privacy rights, HIPAA violations, and the intersection of state and federal laws further contribute to the ongoing discussions surrounding the use of medical marijuana in the workplace.\n',
│   'Question 3: 4. Are there specific legal cases related to medical marijuana use and employment issues?\nAnswer: Yes, there have been legal cases related to medical marijuana use and employment issues in various states.\n',
│   'Question 4: 5. How do different states handle the intersection of medical marijuana and workplace regulations?\nAnswer: Different states handle the intersection of medical marijuana and workplace regulations differently. Some states have specific laws protecting medical marijuana patients from discrimination in the workplace, while others allow employers to enforce their own drug policies, which may include drug testing and potential consequences for employees who test positive for marijuana, even if they are medical marijuana patients. It is important for individuals to be aware of the laws in their specific state regarding medical marijuana and workplace regulations to understand their rights and potential risks.\n'
]

final_query


'Here is the question you need to answer:\n\n    \n --- \n What are the legal issues involving medical marijuana and employment?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in Markdown with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: 1. What legal implications exist regarding medical marijuana and workplace policies?\nAnswer: Workplace policies can supersede an individual\'s possession of a medical marijuana card, potentially leading to conflicts between state laws allowing medical marijuana use and company policies prohibiting it. Employers may enforce zero-tolerance drug policies, even in states where medical marijuana is legal, and this can impact an employee\'s job status. Additionally, there may be legal implications related to discrimination against medical marijuana patients in the hiring process, as seen in the case of the individual in New York who was refused employment after disclosing their medical marijuana card.\n\n\nQuestion 1: 2. How does medical marijuana affect employment laws?\nAnswer: Medical marijuana can impact employment laws differently depending on the state. It is important to consider the specific regulations in each state regarding medical marijuana use, drug testing policies, and workplace discrimination laws. Employers may have varying approaches to medical marijuana use by employees, especially in relation to drug testing, hiring decisions, and workplace policies.\n\n\nQuestion 2: 3. What are the current debates on using medical marijuana in the workplace?\nAnswer: The current debates on using medical marijuana in the workplace revolve around the conflict between state laws legalizing medical marijuana and company policies that maintain zero tolerance drug policies. Issues such as whether having a medical marijuana card protects an employee from repercussions at work, the impact on safety-sensitive positions, and the potential for discrimination against medical marijuana patients in the hiring process are key points of contention in these debates. Additionally, concerns about privacy rights, HIPAA violations, and the intersection of state and federal laws further contribute to the ongoing discussions surrounding the use of medical marijuana in the workplace.\n\n\nQuestion 3: 4. Are there specific legal cases related to medical marijuana use and employment issues?\nAnswer: Yes, there have been legal cases related to medical marijuana use and employment issues in various states.\n\n\nQuestion 4: 5. How do different states handle the intersection of medical marijuana and workplace regulations?\nAnswer: Different states handle the intersection of medical marijuana and workplace regulations differently. Some states have specific laws protecting medical marijuana patients from discrimination in the workplace, while others allow employers to enforce their own drug policies, which may include drug testing and potential consequences for employees who test positive for marijuana, even if they are medical marijuana patients. It is important for individuals to be aware of the laws in their specific state regarding medical marijuana and workplace regulations to understand their rights and potential risks.\n \n --- \n\n\n    Here is additional context relevant to the question:\n\n    \n --- \n I\'m a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.\n\

# Legal Issues Involving Medical Marijuana and Employment

Medical marijuana and employment law is a complex and evolving area. Below are key points addressing the legal issues that arise when medical marijuana intersects with the workplace:

1. **State vs. Federal Law**: Marijuana remains illegal under federal law, classified as a Schedule I substance. This federal prohibition can affect employment, especially in federally regulated industries or federal government jobs, despite state laws legalizing medical marijuana.

2. **Workplace Drug Policies**: Employers may have drug-free workplace policies that prohibit the use of marijuana, including medical marijuana. These policies can often be enforced even in states where medical marijuana is legal, potentially leading to termination or disciplinary action against employees who use medical marijuana.

3. **Safety-Sensitive Positions**: Employers have a legitimate interest in maintaining safety in the workplace. Employees in safety-sensitive positions may be subject to stricter drug use policies, and the use of medical marijuana may not be accommodated, even if it is legal at the state level.

4. **Disability Discrimination**: Some states have laws that protect medical marijuana users from discrimination based on their status as a patient. However, the Americans with Disabilities Act (ADA) does not require employers to accommodate the use of substances that are illegal under federal law.

5. **Drug Testing**: Employers may conduct drug testing, including pre-employment, random, and post-accident testing. A positive test for marijuana can lead to adverse employment actions. The role of medical marijuana cards in drug testing varies by state and employer policy.

6. **HIPAA and Privacy Concerns**: The disclosure of medical marijuana use may raise privacy concerns. Under the Health Insurance Portability and Accountability Act (HIPAA), medical information should be kept confidential, but this does not prevent employers from taking action based on drug test results.

7. **Impairment at Work**: Employers are not required to tolerate on-the-job impairment, even if the employee is a registered medical marijuana user. Employees must still meet performance standards and conduct themselves appropriately in the workplace.

8. **Accommodations**: Employers may be required to consider reasonable accommodations for medical marijuana users, but this does not extend to allowing marijuana use at work or excusing impairment during work hours.

9. **Legal Precedents**: There have been legal cases where employees have challenged termination or discrimination based on medical marijuana use. The outcomes of these cases can vary significantly by state and the specific circumstances involved.

10. **Employment Contracts and Union Agreements**: Individual employment contracts or collective bargaining agreements may have specific provisions regarding drug use and testing that can impact medical marijuana users.

11. **State-Specific Protections**: Some states have enacted laws that provide specific protections for medical marijuana patients, including protections against employment discrimination. It is crucial for employees and employers to be aware of the laws in their particular state.

12. **Employer's Business Interests**: Employers may argue that accommodating medical marijuana use could lead to increased costs, safety risks, or conflicts with other employees or customers, which can influence their policies and actions regarding medical marijuana.

13. **Changing Legal Landscape**: As more states legalize medical and recreational marijuana, the legal landscape continues to change. Employers and employees must stay informed about new laws and court decisions that could affect their rights and obligations.

It is important for both employers and employees to seek legal advice to navigate the complexities of medical marijuana in the workplace, as the interplay between state laws, federal law, and company policies can create a challenging legal environment.

In [38]:
query_text = """What are the legal issues involving medical marijuana and employment?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in Markdown with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=False)
final_res = await mqe.aquery(query_text)
display(Markdown(final_res.text))

0. query


'What legal challenges are present in the intersection of medical marijuana and employment?'

0. sub_query_qa_pairs


[]

sub_query


...

[
│   "Question 0: What legal challenges are present in the intersection of medical marijuana and employment?\nAnswer: The intersection of medical marijuana and employment presents several legal challenges due to the conflicting federal and state laws, as well as varying company policies. Here are some of the key legal challenges:\n\n1. **Federal vs. State Law**: Marijuana remains illegal under federal law as a Schedule I controlled substance, which creates a conflict with states that have legalized medical marijuana. This discrepancy can affect employment, especially in federal jobs or positions that require federal licensing or contracts, where federal law may take precedence.\n\n2. **Workplace Drug Policies**: Many employers maintain strict drug-free workplace policies, which can include zero-tolerance for drug use, including medical marijuana. These policies may be in place for safety reasons, especially in safety-sensitive positions, and can result in the termination of employees who test positive for marijuana, regardless of their medical marijuana patient status.\n\n3. **Disability Discrimination**: Employees using medical marijuana for a disability may face challenges if their employer's actions could be considered discriminatory. The Americans with Disabilities Act (ADA) does not protect employees who use illegal drugs, but there is a gray area when it comes to state-legal medical marijuana. Some states have laws protecting medical marijuana patients from employment discrimination, but these protections vary widely and are not universally recognized.\n\n4. **Drug Testing**: Legal issues arise around drug testing for marijuana, particularly because THC, the psychoactive component in marijuana, can remain in a person's system for weeks after use, making it difficult to determine impairment at work. Employers may face challenges in balancing the need for drug testing with the rights of employees who are legally using medical marijuana.\n\n5. **Accommodation Requests**: Employees who are medical marijuana patients may request accommodations, such as exemptions from drug testing or permission to use medical marijuana outside of work hours. Employers must consider these requests and determine if they can provide reasonable accommodations without causing undue hardship to the business.\n\n6. **Safety Concerns**: Employers in industries with safety-sensitive positions may face legal challenges when trying to reconcile employee rights with public safety. For example, employees who operate heavy machinery or perform tasks that require full cognitive abilities may pose a risk if under the influence of marijuana, even if used for medical purposes.\n\n7. **Privacy Issues**: There are also privacy concerns related to medical marijuana use. Disclosing one's status as a medical marijuana patient may be necessary to explain a positive drug test, but this can raise issues related to medical privacy and the potential for discrimination based on health conditions.\n\n8. **Workers' Compensation and Unemployment Benefits**: The use of medical marijuana can complicate claims for workers' compensation and unemployment benefits. If an employee is injured on the job and tests positive for marijuana, it may affect their eligibility for workers' compensation. Similarly, if an employee is terminated for violating a drug policy, they may be denied unemployment benefits.\n\n9. **State-Specific Protections**: Some states have enacted specific protections for medical marijuana patients, prohibiting discrimination based on an individual's status as a medical marijuana patient. However, these protections are not uniform across all states, and the legal landscape is continually evolving.\n\n10. **Contractual Agreements and Employment Contracts**: Employment contracts may have specific clauses related to drug use and termination for cause. These clauses must be carefully reviewed to ensure they do not conflict with state laws protecting medical marijuana

sub_query


'Here is the question you need to answer:\n\n    \n --- \n What are the implications of medical marijuana legalization on workplace policies and regulations? \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What legal challenges are present in the intersection of medical marijuana and employment?\nAnswer: The intersection of medical marijuana and employment presents several legal challenges due to the conflicting federal and state laws, as well as varying company policies. Here are some of the key legal challenges:\n\n1. **Federal vs. State Law**: Marijuana remains illegal under federal law as a Schedule I controlled substance, which creates a conflict with states that have legalized medical marijuana. This discrepancy can affect employment, especially in federal jobs or positions that require federal licensing or contracts, where federal law may take precedence.\n\n2. **Workplace Drug Policies**: Many employers maintain strict drug-free workplace policies, which can include zero-tolerance for drug use, including medical marijuana. These policies may be in place for safety reasons, especially in safety-sensitive positions, and can result in the termination of employees who test positive for marijuana, regardless of their medical marijuana patient status.\n\n3. **Disability Discrimination**: Employees using medical marijuana for a disability may face challenges if their employer\'s actions could be considered discriminatory. The Americans with Disabilities Act (ADA) does not protect employees who use illegal drugs, but there is a gray area when it comes to state-legal medical marijuana. Some states have laws protecting medical marijuana patients from employment discrimination, but these protections vary widely and are not universally recognized.\n\n4. **Drug Testing**: Legal issues arise around drug testing for marijuana, particularly because THC, the psychoactive component in marijuana, can remain in a person\'s system for weeks after use, making it difficult to determine impairment at work. Employers may face challenges in balancing the need for drug testing with the rights of employees who are legally using medical marijuana.\n\n5. **Accommodation Requests**: Employees who are medical marijuana patients may request accommodations, such as exemptions from drug testing or permission to use medical marijuana outside of work hours. Employers must consider these requests and determine if they can provide reasonable accommodations without causing undue hardship to the business.\n\n6. **Safety Concerns**: Employers in industries with safety-sensitive positions may face legal challenges when trying to reconcile employee rights with public safety. For example, employees who operate heavy machinery or perform tasks that require full cognitive abilities may pose a risk if under the influence of marijuana, even if used for medical purposes.\n\n7. **Privacy Issues**: There are also privacy concerns related to medical marijuana use. Disclosing one\'s status as a medical marijuana patient may be necessary to explain a positive drug test, but this can raise issues related to medical privacy and the potential for discrimination based on health conditions.\n\n8. **Workers\' Compensation and Unemployment Benefits**: The use of medical marijuana can complicate claims for workers\' compensation and unemployment benefits. If an employee is injured on the job and tests positive for marijuana, it may affect their eligibility for workers\' compensation. Similarly, if an employee is terminated for violating a drug policy, they may be denied unemployment benefits.\n\n9. **State-Specific Protections**: Some states have enacted specific protections for medical marijuana patients, prohibiting discrimination based on an individual\'s status as a medical marijuana patient. However, these protections are not uniform across all states, and the legal landscape is continually evolving.\n\n10. **Contractual

final_query


'Here is the question you need to answer:\n\n    \n --- \n What are the legal issues involving medical marijuana and employment?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in Markdown with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What legal challenges are present in the intersection of medical marijuana and employment?\nAnswer: The intersection of medical marijuana and employment presents several legal challenges due to the conflicting federal and state laws, as well as varying company policies. Here are some of the key legal challenges:\n\n1. **Federal vs. State Law**: Marijuana remains illegal under federal law as a Schedule I controlled substance, which creates a conflict with states that have legalized medical marijuana. This discrepancy can affect employment, especially in federal jobs or positions that require federal licensing or contracts, where federal law may take precedence.\n\n2. **Workplace Drug Policies**: Many employers maintain strict drug-free workplace policies, which can include zero-tolerance for drug use, including medical marijuana. These policies may be in place for safety reasons, especially in safety-sensitive positions, and can result in the termination of employees who test positive for marijuana, regardless of their medical marijuana patient status.\n\n3. **Disability Discrimination**: Employees using medical marijuana for a disability may face challenges if their employer\'s actions could be considered discriminatory. The Americans with Disabilities Act (ADA) does not protect employees who use illegal drugs, but there is a gray area when it comes to state-legal medical marijuana. Some states have laws protecting medical marijuana patients from employment discrimination, but these protections vary widely and are not universally recognized.\n\n4. **Drug Testing**: Legal issues arise around drug testing for marijuana, particularly because THC, the psychoactive component in marijuana, can remain in a person\'s system for weeks after use, making it difficult to determine impairment at work. Employers may face challenges in balancing the need for drug testing with the rights of employees who are legally using medical marijuana.\n\n5. **Accommodation Requests**: Employees who are medical marijuana patients may request accommodations, such as exemptions from drug testing or permission to use medical marijuana outside of work hours. Employers must consider these requests and determine if they can provide reasonable accommodations without causing undue hardship to the business.\n\n6. **Safety Concerns**: Employers in industries with safety-sensitive positions may face legal challenges when trying to reconcile employee rights with public safety. For example, employees who operate heavy machinery or perform tasks that require full cognitive abilities may pose a risk if under the influence of marijuana, even if used for medical purposes.\n\n7. **Privacy Issues**: There are also privacy concerns related to medical marijuana use. Disclosing one\'s status as a medical marijuana patient may be necessary to explain a positive drug test, but this can raise issues related to medical privacy and the potential for discrimination based on health conditions.\n\n8. **Workers\' Compensation and Unemployment Benefits**: The use of medical marijuana can complicate claims for workers\' compensation and unemployment benefits. If an employee is injured on the job and tests positive for marijuana, it may affect their eligibility for workers\' compensation. Similarly, if an employee is terminated for violating a drug policy, they may be denied unemployment benefits.\n\n9. **State-Specific Protections**: Some states have enacted specific protections for medical marijuana patients, prohibiting discrimination based on an individual\'s status

In [40]:
display(Markdown(final_res.text))

# Key Legal Issues Involving Medical Marijuana and Employment

The intersection of medical marijuana legalization and employment law presents a complex array of legal issues for both employers and employees. Below are key points that outline these legal challenges:

- **Federal vs. State Law Conflicts**: Marijuana is illegal under federal law but legal for medical use in many states. This creates a conflict for employers who must navigate compliance with both sets of laws, particularly if they are federal contractors or operate in federally regulated industries.

- **Workplace Drug Policies**: Employers often have drug-free workplace policies that prohibit the use of all drugs, including medical marijuana. These policies must be balanced with state laws that may offer protections to medical marijuana users.

- **Disability Discrimination**: The Americans with Disabilities Act (ADA) does not protect the use of federally illegal drugs, but state laws may require employers to accommodate the underlying medical condition for which an employee is using medical marijuana.

- **Drug Testing**: Employers may conduct drug testing, but the presence of THC in a drug test does not necessarily indicate current impairment. State laws may protect medical marijuana patients from adverse employment actions based solely on a positive drug test.

- **Safety-Sensitive Positions**: Employers have a duty to maintain a safe work environment, which can be at odds with an employee's legal use of medical marijuana, especially in positions that are considered safety-sensitive.

- **Privacy and Confidentiality**: Employers must handle information about an employee's medical marijuana use confidentially and in compliance with privacy laws.

- **Workers' Compensation and Unemployment Benefits**: The use of medical marijuana can affect claims for workers' compensation and unemployment benefits, depending on state law and the circumstances of the drug use in relation to the workplace.

- **State-Specific Protections**: Some states have enacted specific protections for medical marijuana patients, prohibiting discrimination based on an individual's status as a medical marijuana patient. Employers in these states must be aware of and comply with these protections.

- **Employment Contracts**: Employment contracts may include clauses related to drug use and termination for cause. These clauses must be reviewed to ensure they do not conflict with state laws protecting medical marijuana patients.

- **Legal Challenges and Litigation**: Employers and employees may find themselves involved in legal disputes over the use of medical marijuana and its impact on employment. Both parties must stay informed about the latest legal developments and seek legal advice when necessary.

- **Reasonable Accommodation**: In some states, employers may be required to consider reasonable accommodations for employees who are qualified medical marijuana patients, as long as the accommodation does not cause undue hardship or pose a safety risk.

- **Training and Communication**: Employers should provide training to HR personnel and managers on how to handle medical marijuana issues in the workplace and clearly communicate company policies to employees.

- **Federal Contracts and Regulations**: Employers with federal contracts must maintain compliance with federal drug-free workplace requirements, which can be at odds with state medical marijuana laws.

These points highlight the multifaceted legal issues that arise when medical marijuana intersects with employment law. Employers and employees must navigate these challenges carefully to ensure legal compliance and protect their respective rights and interests.